# Options At Expiration

#### Step 1. Notebook will refer to the csv file with the list of stock symbols

In [1]:
import csv

STOCKS_FILE_NAME = 'OptionsAtExpirationQuotes.csv'

def get_stocks():
    with open(STOCKS_FILE_NAME, 'r') as f:
        reader = csv.reader(f, delimiter=',', quotechar='"')
        stocks = [stock.strip() for row in reader for stock in row]
    return stocks

get_stocks()

['AAPL', 'NFLX', 'BAC', 'NVDA', 'LUV', 'FB', 'AMZN']

#### Step 2. Notebook will scan options for these stock symbols (around 100 symbols) to identify large option transactions

In [2]:
import queue
import requests
import threading


def load_data(stocks, time, friday):
    
    num_worker_threads = 100
    
    def worker():
        while True:
            args = q.get()
            if args is None:
                break
            get_transactions(q, r, *args)  # <-- see definition of this function below
            q.task_done()

    q = queue.Queue() # queue with arguments for the 'get_transactions' function
    r = queue.Queue() # queue with raw responses

    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for stock in stocks:
        q.put((stock, friday))

    # block until all tasks are done
    q.join()

    # stop workers
    for i in range(num_worker_threads):
        q.put(None)

    for t in threads:
        t.join()
        
    save_rows(r, time)
    
    
def get_transactions(queue, resp_queue, stock, friday):     
    url = "https://query1.finance.yahoo.com/v7/finance/options/{}".format(stock)
    try:
        page = requests.get(url)
    except Exception as e:
        print(e)
    else:
        try:
            response = page.json()
        except Exception as e:
            print(e)
        else:
            if response['optionChain']['error']: # if any error
                print(response['optionChain']['error'])

            else:            
                results = response['optionChain']['result']
                for result in results:
                    nearest_day = dt.fromtimestamp(result['expirationDates'][0]).date()
                    if nearest_day <= friday:
                        price = result['quote']['regularMarketPrice']
                        for option in result['options']:
                            resp_queue.put(
                                (stock, price, option['calls'], option['puts'])
                            )                

In [3]:
# Functions which take response from the response queue and put to the 'largest_transactions' list
def save_rows(q, time):
    while True:
        try:
            stock, price, calls, puts = q.get(block=False)
        except queue.Empty:
            break
        else:
            if stock not in this_week_stocks:
                this_week_stocks.append(stock)
            for call in calls:
                save_row(stock, price, 'Calls', time, call)
            for put in puts:
                save_row(stock, price, 'Puts', time, put)

def save_row(s, price, t, time, raw):
    contact = raw['contractSymbol']    
    previous_volume = transaction_volumes.get(contact)
    volume = raw['volume']
    if previous_volume is None:
        transaction_volumes[contact] = volume
    else:         
        volume_diff = volume - previous_volume
        if volume_diff > 0:            
            transaction_volumes[contact] = volume
            
            last_price = raw['lastPrice']
            strike = raw['strike']
            ask = raw['ask']
            bid = raw['bid']
            if s not in largest_transactions:
                largest_transactions[s] = []
            largest_transactions[s].append(
                dict(
                    stock=s,
                    price=price,
                    type=t,
                    time=time,
                    expiration=dt.fromtimestamp(raw['expiration']),
                    strike=strike,
                    contact=contact,
                    last_price=last_price,
                    bid=bid,
                    ask=ask,
                    volume=volume,
                    open_interest=raw['openInterest'],
                    implied_volatility=raw['impliedVolatility'],
                    volume_diff=volume_diff,
                    cost=volume_diff * 100 * last_price,
                    strike_div_price=(strike / price - 1) * 100,
                    last_price_position="{:.0f}".format((1 - (ask-last_price) / (ask-bid)) * 100) if ask != bid else None
                )
            )

#### Step 3. Notebook will show these transactions in the table

In [4]:
# This function will display the table
from IPython.display import clear_output, display, HTML

headers = (
    "Stock", "Share $", "Options", "Volume", "Open Interest", "Strike", "S / S", 
    "Expiration", "Last price", "Bid", "Ask", "Position",  "Value", "Time of trade",
)
row = """
<td>{stock}</td><td>{price}</td><td>{type}</td><td>{volume_diff}</td><td>{open_interest}</td>
<td>{strike}</td><td>{strike_div_price:.2f}%</td><td>{expiration:%b %d, %Y}</td>
<td>{last_price}</td><td>{bid}</td><td>{ask}</td><td>{last_price_position}%</td>
<td>${cost:,.0f}</td><td>{time:%I:%M%p}</td>
"""

def display_data():    
    table_rows = (row.format(**row_data) for rows in largest_transactions.values() for row_data in rows)           
    clear_output()
    display(
        HTML(
            '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format(
                "</th><th>".join(headers),
                "</tr><tr>".join(table_rows)
            )
        )
    ) 

In [5]:
from datetime import datetime as dt, timedelta as td
from collections import OrderedDict
from pytz import timezone
from time import sleep
import heapq

tz = timezone('America/Los_Angeles')
STOCKS_FILE_NAME = 'OptionsAtExpirationQuotes.csv'
TOP_ITEMS_COUNT = 20
all_stocks = get_stocks()
this_week_stocks = []

last_time = None
count = 1
largest_transactions = {}  # here we will store top entries
transaction_volumes = {}  # latest volumes for transactions

In [ ]:
# The main proccess loop
while True:    
    now = dt.now(tz=tz)
    start = now.replace(hour=6, minute=30, second=0, microsecond=0)
    stop = start.replace(hour=13, minute=0)
    weekday = now.isoweekday()  # 1 - monday, 5 - friday
    
    # refresh this week stocks every monday
    if last_time and last_time.isoweekday() != weekday:
        print('Drop this week stocks')
        this_week_stocks = []
    
    if weekday < 6 and start <= now <= stop: # proper time and day for getting data
        
        if last_time: # no more often than once per minute
            diff = 60 - (now - last_time).seconds            
            if diff > 0:
                sleep(diff) 
        
        days_to_friday = 5 - weekday
        friday = now.date()
        if days_to_friday:
            friday += td(days=days_to_friday)        
        
        load_data(this_week_stocks or all_stocks, now, friday)
        
        # get top transactions
        for k, v in largest_transactions.items():
            largest_transactions[k] = heapq.nlargest(TOP_ITEMS_COUNT, v, key=lambda i: i['cost'])
            
        # sort stocks by cost sum
        largest_transactions = OrderedDict(
            sorted(largest_transactions.items(), key=lambda t: sum(i['cost'] for i in t[1]), reverse=True)
        )        
        display_data()  
        
        # some statistics
        print("prev update", last_time)
        print("last update", now)         
        print("update time", dt.now(tz=tz) - now)
        print("iterations", count)  
        print(this_week_stocks)
        count += 1        
        last_time = now 
    else:
        
        if weekday < 6 and now < start:
            diff = start - now            
        else:
            largest_transactions = {} # drop transactions
            
            days = 8 - weekday if weekday > 5 else 1                
            next_start = start + td(days=days)
            diff = next_start - now
            
        print("time to next start:", diff)
        sleep(diff.seconds + diff.microseconds * 0.1 ** 6)